In [1]:
import pandas as pd
import numpy as np
import os
import glob
import sys

In [2]:
#path to folder with all subjects data

path = '/media/data/HCPAging/data/RestRecommended/fmriresults01/'
#create output dir
os.makedirs('/media/data/HCPAging/data/3T_REST')
path_out = '/media/data/HCPAging/data/3T_REST/'

In [3]:
#read subjects
subjects = sorted([name.split('/')[-1] for name in glob.glob(path+'*_V1_MR')])
#print(len(subjects))

In [4]:
#read rest types
rest_types = sorted(os.listdir(path+subjects[0]+'/MNINonLinear/Results/'))
print(rest_types)

['rfMRI_REST', 'rfMRI_REST1_AP', 'rfMRI_REST1_PA', 'rfMRI_REST2_AP', 'rfMRI_REST2_PA']


### Movements

In [5]:
#Assembling 4 movements file into one
dct_mov={}
for rtype in rest_types[1:]: #for separate single  scans folders
    vec = np.array([], dtype=float)
    nms = []
    rsubjs = sorted([i.split('/')[-4] for i in glob.glob(path + '*/*/*/'+rtype)])
    for subject in rsubjs:
        file = glob.glob(path + subject +'/*/*/'+rtype+'/Movement_RelativeRMS_mean*')
        #print(file)
        if os.path.isfile(file[0]) == True:
            #print('ok', subject, rtype)
            nms += [subject]
            vec = np.append(vec, np.loadtxt(file[0]))
        else:
            print('issue')
    dct_mov[rtype] = pd.Series(vec, index=nms)
        
df_mov = pd.DataFrame(dct_mov)        

In [6]:
#Save table without NA
df_mov.dropna(axis=0).to_csv(path_out+'movements_table_4runs_REST_without_nan.csv')
df_mov.dropna(axis=0).T.mean().to_csv(path_out+'movement_table_averaged_4runs.csv', header=False)

### CIFTI files

In [7]:
#subjects without nan
subjects = sorted(df_mov.dropna(axis=0).index)

In [8]:
len(subjects)

720

In [9]:
#atlases
parc='/media/DataD800/Alina/atlases/Q1-Q6_RelatedValidation210.CorticalAreas_dil_Final_Final_Areas_Group_Colors.32k_fs_LR.dlabel.nii'
mask_fs_path='/media/DataD800/Alina/atlases/FS_subcort'
gla_index = np.loadtxt('/media/DataD800/Alina/atlases/gla_index.txt', dtype=str)

In [18]:
#create folder for intermediate files storing
os.makedirs(path_out+'MSMAll/cort_MSMAll_av_Glasser')
os.mkdir(path_out+'MSMAll/cort_MSMAll_av_Glasser_txt')
os.mkdir(path_out+'MSMAll/subcort_MSMAll_con')
os.mkdir(path_out+'MSMAll/subc_FS_MSMAll_txt')

In [11]:
for subject in subjects:
    if os.path.isfile(path+subject+'/MNINonLinear/Results/'+rest_types[0]+'/rfMRI_REST_Atlas_MSMAll_hp0_clean.dtseries.nii') == False:
        print(subject, 'does not have rfMRI file!')

##### launch file creation through cmd line

In [20]:
#extract surface, parcelate it to gla atlas, save txt, extract subcortex as nii
# split subcortex to structures, and extract average timecouses from structure
#import time
for subject in subjects:
    #apply Glasser altas to surface
    cmd1 = 'wb_command -cifti-parcellate ' + path+subject+'/MNINonLinear/Results/'+rest_types[0]+'/rfMRI_REST_Atlas_MSMAll_hp0_clean.dtseries.nii ' + parc + ' COLUMN '+ path_out+'MSMAll/cort_MSMAll_av_Glasser/'+subject+'.MSMAll.aver_parc.ptseries.nii' + ' -method MEAN'
    #save to txt
    cmd2 = 'wb_command -cifti-convert -to-text '+ path_out+'MSMAll/cort_MSMAll_av_Glasser/'+subject+'.MSMAll.aver_parc.ptseries.nii ' + path_out+'MSMAll/cort_MSMAll_av_Glasser_txt/'+subject+'.MSMAll.aver_parc.txt'
    #separate subcortex as nifti file
    cmd3 = 'wb_command -cifti-separate ' + path+subject+'/MNINonLinear/Results/'+rest_types[0]+'/rfMRI_REST_Atlas_MSMAll_hp0_clean.dtseries.nii ' + ' COLUMN  ' + ' -volume-all ' + path_out+'MSMAll/subcort_MSMAll_con/'+subject+'.MSMAll.con_subc.nii'
    os.system(cmd1)
    os.system(cmd2)
    os.system(cmd3)
    os.mkdir(path_out+'MSMAll/subc_FS_MSMAll_txt/'+subject)
    #apply masks to nifti subcortex and extract txt
    for mask in sorted(os.listdir(mask_fs_path)):
        cmd4 = 'fslmeants -i ' + path_out+'MSMAll/subcort_MSMAll_con/'+subject+'.MSMAll.con_subc.nii' + ' -m ' + mask_fs_path+'/'+mask + ' -o ' + path_out+'MSMAll/subc_FS_MSMAll_txt/'+subject+'/'+subject+'.FS_subcort.'+mask.split('.')[0]+'.mean.txt'
        os.system(cmd4)
    #time.sleep(35)    

##### Assembling scv tables 

In [12]:
dct_group_fc = {}
os.mkdir(path_out+'timecourse_tables')
os.mkdir(path_out+'corrmatrix_tables_Z') #for r-to-z tables

for subject in subjects:
    #reading file with Glasser atlas parcellation
    df_rcort = pd.read_csv(glob.glob(path_out+'MSMAll/cort_MSMAll_av_Glasser_txt/'+subject+"*")[0], 
                           sep='\t', header=None)
    df_rcort.index = gla_index
    
    #reading subcortex files
    dct_rsubc = {}
    for file in sorted(glob.glob(path_out+'MSMAll/subc_FS_MSMAll_txt/'+subject+'/*')):
        dct_rsubc[str(file.split('.')[-3])] = np.loadtxt(file)
    df_subc = pd.DataFrame(dct_rsubc).T

    #combine cort and subc into one table
    df_rest = pd.concat([df_rcort , df_subc], axis=0)

    #create correlation matrix and do r-to-z
    df_rest_corr = df_rest.T.corr()
    df_rest_corr_z = np.arctanh(df_rest_corr) #r-to-z

    #transform matrix to string for the subjec
    dfg = df_rest_corr_z.where(np.triu(np.ones(df_rest_corr_z.shape), k=1).astype(bool)).stack().reset_index()
    indx = [i+'_&_'+j for i,j in zip(dfg['level_0'], dfg['level_1'])]
    fc_string = pd.Series(np.array(dfg[0]),index=indx)
    dct_group_fc[subject] = fc_string #dictionary for group table

    #save individual files
    df_rest.to_csv(path_out+'timecourse_tables/'+subject+'_rest_timecourse.csv')
    df_rest_corr_z.to_csv(path_out+'corrmatrix_tables_Z/'+subject+'_rest_fc_z.csv')

#save group FC table
df_group_fc = pd.DataFrame(dct_group_fc).T
df_group_fc.to_csv(path_out+'FC_rest_group_z.csv')